# Suite2p notebook
This file is read only. Please make a copy in your home directory before modifying this file.

In [7]:
import sys
sys.path.insert(1, 'E:/Users/samsoon.inayat/T_Drive/GitHub/Toolbox_2P/Suite2P_Wrapper/extras')

from parse_xml import parse_xml
from raw2h5 import raw2h5
import logging
import os
import glob
import argparse

## Setup input
Setting the ```run_mode``` variable will control which parts of the pipeline are run.
- all: Run the entire pipeline
- suite2p: Run only the registration/cell extraction part
- deconv: Run the deconvolution algorithm from the MATLAB version of suite2p on the extracted cells

Either insert the path to a file in the ```params_file``` variable, or fill your entries in the db variable below.
Make sure your input/output paths are created before running this.

In [ ]:
logging.basicConfig(format="%(asctime)s %(relativeCreated)12d [%(filename)s:%(funcName)15s():%(lineno)s] [%(process)d] %(message)s", 
                    datefmt='%Y-%m-%d %H:%M:%S', level=logging.DEBUG)
run_mode = 'all' # Use 'suite2p' to just run the registration/cell extraction, 'deconv' for just the deconvolution or 'all' for everything
params_file = '' # Place path to params.py here if you want to use a separate file.
if os.path.isfile(params_file):
    with open(params_file) as f:
        code = compile(f.read(), params_file, 'exec')
        exec(code)
else:
    db = []
    
    db.append({
        "mouse_name": "mPFC10",
        "date": "2019_06_16",
        "expts": [1, 2],
        "raw_dir": "/global/scratch/karim/data/",
        "tif_dir": "/global/scratch/karim/tif/",
        "results_dir": "/global/scratch/karim/results",
        "ops": {
            #"save_mat": False,
        }
    })

## Run the actual pipeline

In [ ]:
for i in db:
    comb_dir = '_'.join(str(x) for x in i['expts'])
    tif_dir = os.path.join(i['tif_dir'], i['mouse_name'], i['date'], comb_dir)
    results_dir = os.path.join(i['results_dir'], i['mouse_name'], i['date'], comb_dir)
    
    if not os.path.isdir(results_dir):
        logging.info(f'Creating results directory {results_dir}')
        os.makedirs(results_dir)
    
    if 'abf_dir' in i:
        abf_dir = [ os.path.join(i['abf_dir'], i['mouse_name'], i['date'] + "_" + str(x) + ".abf") for x in i['expts'] ]
    else:
        abf_dir = None
    print(f'Running expt {i}')
    print(f'Temp directory: {tif_dir}')
    print(f'Results directory: {results_dir}')
    i['in_dir'] = os.path.join(i['raw_dir'], i['mouse_name'], i['date'])
    offset = 1
    nframes = []
    for j in i['expts']:
        in_dir = os.path.join(i['in_dir'], str(j))
        
        xml_path = os.path.join(in_dir, "Experiment.xml")
        logging.info(f'Parsing Experiment.xml in {xml_path}')
        if os.path.exists(xml_path):
            (x, y, fr, num_frames, num_planes) = parse_xml(xml_path)
            nframes.append(num_frames)
        else:
            raise ValueError(f"Experiment.xml does not exist at {xml_path}")
            
    if run_mode == 'suite2p' or run_mode == 'all':
        from run_suite2p import run_suite2p
        ops = run_suite2p(i, results_dir, fr/num_planes, num_planes)
    if run_mode == 'deconv' or run_mode == 'all':
        from deconv import do_deconv
        do_deconv(results_dir, num_planes/fr, nframes, i['expts'], abf_dir, nplanes=num_planes)